## 今天根据PyTorch教程中的60min Blitz走第不知道几遭

*今天是星期天哟*

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

### Neural Networks

可以用`torch.nn`包搭建神经网络

现在已经初步了解了`autograd`,`nn`根据`autograd`定义模型并对其求导。一个`nn.Module`包含层*(layers)*,及一个返回输出`output`的方法`forward(input)`

例如，LeNet结构如下：

![jupyter](./mnistLeNet.png)

典型的神经网络训练过程如下：
- 定义具有若干可学习参数（或权重）的神经网络
- 根据数据集*(dataset)*迭代输入
- 神经网络处理输入
- 计算loss值
- 将梯度反向传播回网络的各个参数
- 更新网络权值，通常依据如下规则：`weight = weight - learning_rate * gradient`

##### Define the network

定义网络如下：

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution kernel
        self.conv1=nn.Conv2d(1,6,3)
        self.conv2=nn.Conv2d(6,16,3)
        # an affine operation: y=Wx+b（矩阵乘法）
        self.fc1=nn.Linear(16*6*6,120) #6*6来自于图像维度
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    def forward(self,x):
        # Maxpooling over a (2,2) window
        x=F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        # 若window形状是正方形，则可只指定一个数值（边长）
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    def num_flat_features(self,x):
        size=x.size()[1:]# x[1:]由x[1]到最后一个元素组成
        num_features=1
        for s in size:
            num_features*=s
        return num_features
net=Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


只需要定义`forward`函数，使用`autograd`会自动创建`backward`函数。在`forward`函数中可以使用任意tensor运算

`net.parameters()`返回一个模型的可学习参数

In [2]:
params=list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])
